In [ ]:
import distutils
import distutils.util
import numpy as np
import pandas as pd
import os
import os.path as opath
import ast
import json
from pandas.io.json import json_normalize
import datetime
import pickle
import re
from datetime import datetime
from IPython.core.display import HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<div class="span5 alert alert-info">
<h3>Note book details</h3>

<p> This notebook is for part 3 of 3 in data wrangling for <b>kickstarter success prediction</b> project.</p>

<p> Notes.</p>
<ol>
<li>  The file will filter columns needed for data story</li> 
<li>  The file will rename columns to more descriptive and clear names</li> 
<li>  The following columns are also added/modified 
    <ul>
    <li>exchange rate : exchange rate to get later use for non USD projects</li>
    <li>usd_goal : converted goal amount to USD equivalent for comparision</li>
    <li>state : Converted to title case for better using as label </li>
    <li>pledge_count_ratio : Pledge amount / no of backer</li>
    <li>pledge_goal_ratio : Pledge amount / goal amount received</li>
    <li>pledge_extra_goal_ratio : (Pledge amount- goal amount) / goal amount received</li>
    <li>deadline_days : Deadline date- Project start date</li>
    <li>state_change_days : State change date- Project start date</li>
    <li></li>
    </ul>
</li>     
<li>  The single pickle files created to use further in project . This make easier to use data in later stages</li> 
    
</ol>
</div>

In [ ]:
pd.set_option('display.max_colwidth', -1)
stateSuccess='Successful'
stateFail='Failed'

In [ ]:
# from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))
pickle_file='Jan2017_Jan2018.sa'

In [ ]:
# reading previous pickel file
df_all_months = pickle.load( open( pickle_file, "rb" ) )

#### check all the below when new month added

In [ ]:
# check all the below when new month added
#check project without goal, amount. check if new month added
#df_final_months[(df_final_months.goal<=0 )]
#df_final_months[(df_final_months.pledged<=0 )& (df_final_months.backers_count<=0 )]

# check distinctt state
#df_final_months.state.unique()

# check suspened and exclude
#df_final_months[df_final_months.state=='suspended']

# check canceled and exclude
#df_final_months[df_final_months.state=='canceled']

## Data manipulation helping methods

In [ ]:
def convertCamelCase(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

In [ ]:
def convertTitleCase(name):
    return name.title()

In [ ]:
def convertTitleCase_State(row):
    return convertTitleCase(row)

In [ ]:
def calculate_ratio(numerator, denominator):
    ratio=0
    if(denominator>0).any():
        ratio=numerator/denominator
    return np.round(ratio,3)

In [ ]:
def get_days_difference(date_from, date_to):
    print(date_from-date_to)#.days

In [ ]:
def compare_dates(from_date, to_date):
    date_format = '%Y-%m-%d %H:%M:%S'#'%Y/%m/%d'
    from_date_val = datetime.strptime(from_date, date_format)  
    to_date_val= datetime.strptime(to_date, date_format)  
    #today = datetime.today()
    diff = from_date_val - to_date_val
    return diff.days

In [ ]:
def diff_deadline_dates(row):
    return compare_dates(row.deadline,row.launched_at)

In [ ]:
def diff_state_changed_dates(row):
    return compare_dates(row.state_changed_at,row.launched_at)

In [ ]:
def convertToDateFormat(val):
    date_format = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(val,date_format)

def convertCreateDateToDateFormat(row):
    return convertToDateFormat(row)
def getMonth(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(row.launched_at,date_format).month
    #print (type(row.created_at))
def getYear(row):
    date_format = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(row.launched_at,date_format).year

In [ ]:
def convertUnixTimeToDateTime(input):
        return ( datetime.datetime.fromtimestamp(int(input)).strftime('%Y-%m-%d %H:%M:%S'))

## Seleting & renaming some fields for main dataframe

In [ ]:
#df_final_months=df_all_months[
df_final_months=df_all_months.loc[:,
    ['id',
'Catgegory_val',
'backers_count',
'country',
'location_type',
'created_at_val',
'deadline_val',  
'goal',
'launched_at_val', 
'name',
'pledged',
'spotlight', 
'staff_pick',
'state',
'state_changed_at_val',  
'usd_pledged',
'userid'
]]
_=df_final_months.set_index('id')
_=df_final_months.rename(columns={'id': 'project_id', 'Catgegory_val': 'category',
                                'created_at_val':'created_at','launched_at_val':'launched_at',
                                'state_changed_at_val':'state_changed_at','deadline_val':'deadline'}, inplace=True)

#### Adding and modifyng columns

In [ ]:
# calculating exchange rate and using it to get the USD value for goal for non -USD projects
exchange_rate_ser= df_final_months.usd_pledged/df_final_months.pledged
df_final_months = df_final_months.assign( exchange_rate=pd.to_numeric( exchange_rate_ser.fillna('1')))
df_final_months=df_final_months.assign(usd_goal= lambda row : row.goal * (row.exchange_rate))

In [ ]:
# converting State to title case
df_final_months['state'] = df_final_months['state'].apply(convertTitleCase_State)

In [ ]:
# calculting and adding pledge count ration pledge goal ration and pledge extra goal ration columns
df_final_months=df_final_months.assign(
    pledge_count_ratio=lambda row: calculate_ratio(row.usd_pledged,row.backers_count)
)
df_final_months=df_final_months.assign(
    pledge_goal_ratio=lambda row: calculate_ratio(row.usd_pledged,row.usd_goal))

df_final_months=df_final_months.assign(
    pledge_extra_goal_ratio=lambda row: calculate_ratio((row.usd_pledged-row.usd_goal),row.usd_pledged))

df_final_months['pledge_count_ratio'].fillna(0,inplace=True)
df_final_months['pledge_goal_ratio'].fillna(0,inplace=True)
df_final_months['pledge_extra_goal_ratio'].fillna(0,inplace=True)


In [ ]:
# adding deadline days and state change days 
df_final_months['deadline_days']=df_final_months.apply(diff_deadline_dates,axis=1)
df_final_months['state_change_days']=df_final_months.apply(diff_state_changed_dates,axis=1)

In [ ]:
# use if precise hours is needed , currently not needed
#df_fail_success=df_fail_success.assign(deadline_days_hrs=lambda row : pd.to_datetime(row.deadline)-pd.to_datetime(row.launched_at))
#df_fail_success=df_fail_success.assign(state_change_days_hrs=lambda row : pd.to_datetime(row.state_changed_at)-pd.to_datetime(row.launched_at))

In [ ]:
# setting staff pick as category
_=df_final_months.staff_pick.astype('category')

#### Saving the pickel file to use later

In [ ]:
picklefilename='DataStory_Jan2017_Jan2018.sa'
pickle.dump(df_final_months,open(picklefilename,'wb'))

In [ ]:
# check pickle file
upickle_df = pickle.load(open(picklefilename,"rb"))

In [ ]:
upickle_df.info()